In [1]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    classification_report
)

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [4]:
df_train.head() # 학습용 데이터 살펴보기

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.0,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.0,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.0,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True


In [5]:
df_test.head() # 학습용 데이터 살펴보기

,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,19844,0.00,/ / Brazil,ID,0.073248,47466,End Customer,Enterprise,53.0,NaN,...,LGESP,NaN,1,0,0.001183,0.049840,retail,Electronics & Telco,278,True
1,9738,0.25,400 N State Of Franklin Rd Cloud IT / Johnson...,IT,NaN,5405,End Customer,SMB,NaN,NaN,...,LGEUS,NaN,0,0,0.000013,NaN,transportation,Others,437,True
2,8491,1.00,/ / U.A.E,ID,NaN,13597,Specifier/ Influencer,SMB,NaN,NaN,...,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,True
3,19895,0.50,/ Madison / United States,ID,0.118644,17204,NaN,Enterprise,NaN,NaN,...,LGEUS,more than a year,0,0,0.001183,0.049840,retail,NaN,194,False
4,10465,1.00,/ Sao Paulo / Brazil,ID,0.074949,2329,End Customer,Enterprise,2.0,1.0,...,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,167,True


In [6]:
'''
# customer_country, customer_country.1 날리고 response_orporate만 남기기
'''
drop_col = ['customer_country.1', 'customer_country', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver']
df_train.drop(drop_col, axis=1, inplace=True)
df_test.drop(drop_col, axis=1, inplace=True)

In [7]:
label_columns = list(df_train.columns)

label_columns.remove("is_converted")

In [8]:
'''
columns = label_columns

# 결과 출력
for column in columns:
    # 각 열에 대한 missing value 비율 계산
    missing_value_ratio = df_train[column].isnull().mean()

    # missing value인 row의 'is_converted' column의 값이 True인 비율 계산
    missing_value_converted_ratio = df_train.loc[df_train[column].isnull(), 'is_converted'].mean()

    # non-missing value인 row의 'is_converted' column의 값이 True인 비율 계산
    non_missing_value_converted_ratio = df_train.loc[~df_train[column].isnull(), 'is_converted'].mean()

    print(f"Column: {column}")
    print(f"Missing Value Ratio: {missing_value_ratio}")
    print(f"Missing Value Converted Ratio: {missing_value_converted_ratio}")
    print(f"Non-Missing Value Converted Ratio: {non_missing_value_converted_ratio}")
    '''

'\ncolumns = label_columns\n\n# 결과 출력\nfor column in columns:\n    # 각 열에 대한 missing value 비율 계산\n    missing_value_ratio = df_train[column].isnull().mean()\n\n    # missing value인 row의 \'is_converted\' column의 값이 True인 비율 계산\n    missing_value_converted_ratio = df_train.loc[df_train[column].isnull(), \'is_converted\'].mean()\n\n    # non-missing value인 row의 \'is_converted\' column의 값이 True인 비율 계산\n    non_missing_value_converted_ratio = df_train.loc[~df_train[column].isnull(), \'is_converted\'].mean()\n\n    print(f"Column: {column}")\n    print(f"Missing Value Ratio: {missing_value_ratio}")\n    print(f"Missing Value Converted Ratio: {missing_value_converted_ratio}")\n    print(f"Non-Missing Value Converted Ratio: {non_missing_value_converted_ratio}")\n    '

In [9]:
'''
# 수치형 변수 전체 Standardizaiton, Normalization 실행
'''
# Initialize scalers
scaler_standard = StandardScaler()
scaler_minmax = MinMaxScaler()
columns_to_ST = ["com_reg_ver_win_rate", "historical_existing_cnt","lead_desc_length"]
columns_to_NM = ["ver_win_rate_x", "ver_win_ratio_per_bu"]

# Apply Standardization to each column
for column in columns_to_ST:
    # Standardization
    df_train[column + '_standardized'] = scaler_standard.fit_transform(df_train[[column]])
    df_test[column + '_standardized'] = scaler_standard.fit_transform(df_test[[column]])
    # Drop the original column
    df_train.drop(columns=[column], inplace=True)
    df_test.drop(columns=[column], inplace=True)

# Apply Standardization to each column
for column in columns_to_NM:
    # Normalization
    df_train[column + '_normalized'] = scaler_minmax.fit_transform(df_train[[column]])
    df_test[column + '_normalized'] = scaler_minmax.fit_transform(df_test[[column]])
    # Drop the original column
    df_train.drop(columns=[column], inplace=True)
    df_test.drop(columns=[column], inplace=True)

# Print the first few rows of the DataFrame to check the results
print(df_train.head())

   bant_submit business_unit  customer_idx          customer_type  enterprise  \
0          1.0            AS         32160           End-Customer  Enterprise   
1          1.0            AS         23122           End-Customer  Enterprise   
2          1.0            AS          1755           End-Customer  Enterprise   
3          1.0            AS          4919           End-Customer  Enterprise   
4          1.0            AS         17126  Specifier/ Influencer  Enterprise   

              customer_job                        inquiry_type  \
0               purchasing  Quotation or purchase consultation   
1  media and communication  Quotation or purchase consultation   
2              engineering                 Product Information   
3         entrepreneurship  Quotation or purchase consultation   
4               consulting  Quotation or purchase consultation   

  product_category product_subcategory product_modelname  ... ver_pro  \
0      multi-split                 NaN     

In [10]:
'''
# 개별 column 데이터 결측치 보간
'''
# col_to_fill = ["customer_type", "customer_job", "inquiry_type", "product_subcategory", "product_modelname", "expected_timeline", "business_area"]


'\n# 개별 column 데이터 결측치 보간\n'

In [11]:
'''
# business_area 날리고 2개 피처 추가
'''
def is_hospital(value):
    if value == "hospital & health care":
        return 1
    else:
        return 0

def is_power(value):
    if value == "power plant / renewable energy":
        return 1
    else:
        return 0

# "business_area" 열에서 함수를 적용하여 새로운 열 생성
df_train['is_hospital'] = df_train['business_area'].apply(is_hospital)
df_train['is_power'] = df_train['business_area'].apply(is_hospital)
df_test['is_hospital'] = df_test['business_area'].apply(is_hospital)
df_test['is_power'] = df_test['business_area'].apply(is_hospital)

#df_train.drop(columns=["business_area"], inplace=True)
#df_test.drop(columns=["business_area"], inplace=True)

In [12]:
'''
# expected_timeline 에서 특정 단어 필터링
'''
# 새로운 열을 추가하기 위한 함수 정의
def contains_budget(value):
    if 'budget' in str(value).lower():  # 대소문자 구분없이 'budget'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_budget'] = df_train['expected_timeline'].apply(contains_budget)
df_test['contains_budget'] = df_test['expected_timeline'].apply(contains_budget)

# 새로운 열을 추가하기 위한 함수 정의
def contains_etc(value):
    if 'etc' in str(value).lower():  # 대소문자 구분없이 'budget'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_etc'] = df_train['expected_timeline'].apply(contains_etc)
df_test['contains_etc'] = df_test['expected_timeline'].apply(contains_etc)

# 새로운 열을 추가하기 위한 함수 정의
def contains_hence(value):
    if 'hence' in str(value).lower():  # 대소문자 구분없이 'budget'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_hence'] = df_train['expected_timeline'].apply(contains_hence)
df_test['contains_hence'] = df_test['expected_timeline'].apply(contains_hence)

# 새로운 열을 추가하기 위한 함수 정의
# def contains_system(value):
#     if 'system' in str(value).lower():  # 대소문자 구분없이 'budget'이 포함되어 있는지 확인
#         return 1
#     else:
#         return 0

# # "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
# df_train['contains_system'] = df_train['expected_timeline'].apply(contains_system)
# df_test['contains_system'] = df_test['expected_timeline'].apply(contains_system)

# # 새로운 열을 추가하기 위한 함수 정의
# def contains_closi(value):
#     if 'closi' in str(value).lower():  # 대소문자 구분없이 'budget'이 포함되어 있는지 확인
#         return 1
#     else:
#         return 0

# # "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
# df_train['contains_closi'] = df_train['expected_timeline'].apply(contains_closi)
# df_test['contains_closi'] = df_test['expected_timeline'].apply(contains_closi)

# # 새로운 열을 추가하기 위한 함수 정의
# def contains_any(value):
#     if 'any' in str(value).lower():  # 대소문자 구분없이 'budget'이 포함되어 있는지 확인
#         return 1
#     else:
#         return 0

# # "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
# df_train['contains_any'] = df_train['expected_timeline'].apply(contains_any)
# df_test['contains_any'] = df_test['expected_timeline'].apply(contains_any)

# 새로운 열을 추가하기 위한 함수 정의
def contains_although(value):
    if 'although' in str(value).lower():  # 대소문자 구분없이 'budget'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_although'] = df_train['expected_timeline'].apply(contains_although)
df_test['contains_although'] = df_test['expected_timeline'].apply(contains_although)

# 새로운 열을 추가하기 위한 함수 정의
def contains_more(value):
    if 'more' in str(value).lower():  # 대소문자 구분없이 'budget'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_more'] = df_train['expected_timeline'].apply(contains_more)
df_test['contains_more'] = df_test['expected_timeline'].apply(contains_more)

# 새로운 열을 추가하기 위한 함수 정의
def contains_year(value):
    if 'year' in str(value).lower():  # 대소문자 구분없이 'budget'이 포함되어 있는지 확인
        return 1
    else:
        return 0

# "expected_timeline" 열에서 함수를 적용하여 새로운 열 생성
df_train['contains_year'] = df_train['expected_timeline'].apply(contains_year)
df_test['contains_year'] = df_test['expected_timeline'].apply(contains_year)

In [13]:
'''
def classify_text_with_bert(input_text, categories):
    # Load pre-trained BERT model and tokenizer
    model_name = 'bert-base-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(categories))
    
    # Tokenize input sentence and add padding and special tokens
    if str == type(input):
      inputs = tokenizer(input_text, padding=True, truncation=True, return_tensors="pt")
    else :
      return np.NaN
    
    # Set model to evaluation mode
    model.eval()
    
    # Perform prediction by inputting the sentence into the BERT model
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Convert prediction result to probabilities and determine predicted category
    predicted_probabilities = torch.softmax(outputs.logits, dim=1).squeeze().tolist()
    predicted_category_index = predicted_probabilities.index(max(predicted_probabilities))
    predicted_category = categories[predicted_category_index]
    
    return predicted_category

categories = ["Less than 3 months", "Between 3 months to 6 months", "Between 6 months to 12 months", "More than 12 months"]  # Example category list, modify according to your actual categories

# expected_timeline 컬럼에 classify_text_with_bert 함수 적용하여 새로운 열 생성
df_train['expected_timeline_category'] = df_train['expected_timeline'].apply(lambda x: classify_text_with_bert(x, categories))
df_test['expected_timeline_category'] = df_test['expected_timeline'].apply(lambda x: classify_text_with_bert(x, categories))

df_train.drop(columns=["expected_timeline"], inplace=True)
df_test.drop(columns=["expected_timeline"], inplace=True)
'''


'\ndef classify_text_with_bert(input_text, categories):\n    # Load pre-trained BERT model and tokenizer\n    model_name = \'bert-base-uncased\'\n    tokenizer = BertTokenizer.from_pretrained(model_name)\n    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(categories))\n    \n    # Tokenize input sentence and add padding and special tokens\n    if str == type(input):\n      inputs = tokenizer(input_text, padding=True, truncation=True, return_tensors="pt")\n    else :\n      return np.NaN\n    \n    # Set model to evaluation mode\n    model.eval()\n    \n    # Perform prediction by inputting the sentence into the BERT model\n    with torch.no_grad():\n        outputs = model(**inputs)\n    \n    # Convert prediction result to probabilities and determine predicted category\n    predicted_probabilities = torch.softmax(outputs.logits, dim=1).squeeze().tolist()\n    predicted_category_index = predicted_probabilities.index(max(predicted_probabilities))\n    p

In [14]:
    
# Calculate the most frequent value (mode) for each column
modes = df_train.mode().iloc[0]

# Fill missing (NA) values with the mode
df_train.fillna(modes, inplace=True)

# Calculate the most frequent value (mode) for each column
modes = df_test.mode().iloc[0]

# Fill missing (NA) values with the mode
df_test.fillna(modes, inplace=True)


In [15]:
'''
df_train과 df_test에서 숫자 형식인거 다 str로 바꾸기 (일부제외)
'''

# 변환할 라벨들의 리스트
labels_to_convert = [
    "bant_submit",
    "com_reg_ver_win_rate_standardized",
    "historical_existing_cnt_standardized",
    "id_strategic_ver",
    "it_strategic_ver",
    "idit_strategic_ver",
    "lead_desc_length_standardized",
    "ver_cus",
    "ver_pro",
    "ver_win_rate_x_normalized",
    "ver_win_ratio_per_bu_normalized"
]

# df_test와 df_train에서 라벨들의 데이터 타입을 str로 변환
for label in labels_to_convert:
    if label in df_test.columns:
        df_test[label] = df_test[label].astype(str)
    if label in df_train.columns:
        df_train[label] = df_train[label].astype(str)

df_train['is_converted'] = df_train['is_converted'].astype(int)
df_test['is_converted'] = df_test['is_converted'].astype(int)

In [16]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

In [17]:
cat_features_col = list(x_train.columns)

In [18]:
catboost_model = CatBoostClassifier(
    verbose=0, 
    iterations=1000, 
    depth=6, 
    learning_rate=0.1, 
    loss_function='CrossEntropy', 
    one_hot_max_size=5,
    cat_features=cat_features_col
    )

In [19]:
catboost_model.fit(x_train, y_train)

In [20]:
# 모델을 사용하여 검증 데이터셋에 대한 예측 수행
y_val_pred = catboost_model.predict(x_val)
print (y_val_pred)

y_val_pred = np.array([True if pred == 'True' else False for pred in y_val_pred])

print("Classification Report:\n", classification_report(y_val, y_val_pred))
print("오차행렬:\n", confusion_matrix(y_val, y_val_pred, labels=[True, False]))
print("\n정확도: {:.4f}".format(accuracy_score(y_val, y_val_pred)))
print("정밀도: {:.4f}".format(precision_score(y_val, y_val_pred, labels=[True, False])))
print("재현율: {:.4f}".format(recall_score(y_val, y_val_pred)))
print("F1: {:.4f}".format(f1_score(y_val, y_val_pred, labels=[True, False])))

[0 0 0 ... 0 0 0]
Classification Report:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     10913
           1       0.00      0.00      0.00       947

    accuracy                           0.92     11860
   macro avg       0.46      0.50      0.48     11860
weighted avg       0.85      0.92      0.88     11860

오차행렬:
 [[    0   947]
 [    0 10913]]

정확도: 0.9202
정밀도: 0.0000
재현율: 0.0000
F1: 0.0000


/home/elicer/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/elicer/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/elicer/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/elicer/.local/lib/python3.10/si

In [21]:
x_test = df_test.drop(["is_converted", "id"], axis=1)

test_pred = catboost_model.predict(x_test)

df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred
df_sub['is_converted'] = df_sub['is_converted'].astype(bool)
print(df_sub['is_converted'])

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

0        True
1        True
2       False
3       False
4        True
        ...  
5266    False
5267     True
5268    False
5269    False
5270     True
Name: is_converted, Length: 5271, dtype: bool
